In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
#!pip install bokeh

In [2]:
import glob
import os
import warnings
import numpy as np
from bokeh.io import export_svgs, output_notebook
from bokeh.models import BoxAnnotation, ColumnDataSource, HoverTool
from bokeh.plotting import figure, show
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    average_precision_score,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from utils.measuring_performance import (
    get_prediction,
    plot_confusion_matrix,
    plot_histogram_by_class,
    plot_loss_per_epoch,
    plot_pr_curve,
    plot_roc_curve,
)
from utils.misc import build_files_list, dump_pickle, load_pickle
from utils.sound_utils import extract_signal_features, generate_dataset, load_sound_file

output_notebook()
warnings.filterwarnings("ignore")
np.random.seed(42)

Loading BokehJS ...

In [10]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
#from tensorflow.keras.utils import multi_gpu_model
from tensorflow.python.client import device_lib

tf.random.set_seed(42)

In [11]:
def get_available_gpus():
    local_devices = device_lib.list_local_devices()
    return [x.name for x in local_devices if x.device_type == "GPU"]

The boolean variable `COMBINE_MACHINE_ID` allows you to decide whether to perform modeling separately for each machine ID, or combine all data from the same machine type to perform modeling.

In [12]:
DATA_PATH = "../../data/mimii-anomaly-detection"
IMAGE_PATH = "./img"
MODEL_PATH = "./models"
MERGE_MACHINE_ID = True

os.makedirs(os.path.join(DATA_PATH, "dataset"), exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)

file_paths = sorted(
    glob.glob(DATA_PATH + "/*/*" if MERGE_MACHINE_ID else DATA_PATH + "/*/*/*")
)

# Data Splitting and Preprocessing

In [13]:
file_path = file_paths[0]  # the first machine ID or machine type selected
file_path_split = file_path.split("/")
SUFFIX = "_".join(["", file_path_split[-1], file_path_split[-2]])

if MERGE_MACHINE_ID:
    print(f"db: {file_path_split[-2]}, machine type: {file_path_split[-1]}")

else:
    print(
        f"db: {file_path_split[-3]}, machine type: {file_path_split[-2]}, machine id: {file_path_split[-1]}"
    )
    SUFFIX = "_".join([SUFFIX, file_path_split[-3]])

db: 6dB, machine type: fan


Because unsupervised learning is used, all anomalous labels are assigned to the test set.

In [15]:
normal_files, abnormal_files = build_files_list(root_dir=file_path)
normal_labels = np.zeros(len(normal_files))
abnormal_labels = np.ones(len(abnormal_files))

train_files, test_files, train_labels, test_labels = train_test_split(
    normal_files, normal_labels, train_size=0.8, random_state=42, shuffle=True
)

test_files = np.concatenate((test_files, abnormal_files), axis=0)
test_labels = np.concatenate((test_labels, abnormal_labels), axis=0)

test_indices = np.arange(len(test_files))
np.random.shuffle(test_indices)

test_files = test_files[test_indices]
test_labels = test_labels[test_indices]

print(
    f"Train set has {train_labels.shape[0]} signals including {train_labels.sum():.0f} abnormal signals, \
but test set has {test_labels.shape[0]} signals including {test_labels.sum():.0f} abnormal signals."
)

Train set has 3260 signals including 0 abnormal signals, but test set has 2290 signals including 1475 abnormal signals.


In [16]:
dataset = {
    "train_files": train_files,
    "test_files": test_files,
    "train_labels": train_labels,
    "test_labels": test_labels,
}

for key, values in dataset.items():
    file_name = os.path.join(DATA_PATH, "dataset", key + SUFFIX + ".txt")
    with open(file_name, "w") as f:
        for item in values:
            f.write(str(item) + "\n")

In [18]:
n_fft = 2048
hop_length = 512
n_mels = 64
frames = 5

train_data_path = os.path.join(DATA_PATH, "dataset", "train_data" + SUFFIX + ".pkl")

if os.path.exists(train_data_path):
    print("Train data already exists, loading from file...")
    train_data = load_pickle(train_data_path)

else:
    train_data = generate_dataset(
        train_files, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, frames=frames
    )
    print("Saving train data to disk...")
    dump_pickle(train_data_path, train_data)
    print("Done.")

print(f"Train data has a {train_data.shape} shape.")

Train data already exists, loading from file...
Train data has a (1007340, 320) shape.


# Model Training and Prediction with AutoEncoder

In [19]:
def autoencoder(input_dims, model_name=None):
    input_layer = Input(shape=(input_dims,))
    output = Dense(64, activation="relu")(input_layer)
    output = Dense(64, activation="relu")(output)
    output = Dense(8, activation="relu")(output)
    output = Dense(64, activation="relu")(output)
    output = Dense(64, activation="relu")(output)
    output = Dense(input_dims, activation=None)(output)

    return Model(inputs=input_layer, outputs=output, name=model_name)

In [25]:
from tensorflow.python.client import device_lib
MODEL_NAME = "AutoEncoder"
model = autoencoder(n_mels * frames, model_name=MODEL_NAME)
print(model.summary())

print(f"n_mels = {n_mels}, frames = {frames}")

gpu_count = len(get_available_gpus())
print(f"Available gpus = {gpu_count}")
if gpu_count > 1:
    model = multi_gpu_model(model, gpus=gpu_count)

Model: "AutoEncoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 320)]             0         
                                                                 
 dense_12 (Dense)            (None, 64)                20544     
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 dense_14 (Dense)            (None, 8)                 520       
                                                                 
 dense_15 (Dense)            (None, 64)                576       
                                                                 
 dense_16 (Dense)            (None, 64)                4160      
                                                                 
 dense_17 (Dense)            (None, 320)               

In [26]:
%%time
batch_size = 512
epochs = 200

model.compile(
    optimizer=Adam(learning_rate=1e-03),
    loss="mean_squared_error"
)

history = model.fit(
    train_data,
    train_data,
    batch_size=batch_size,
    epochs=epochs,
    verbose=True,
    # https://keras.io/api/callbacks/early_stopping/
    callbacks=[EarlyStopping(monitor="val_loss", patience=10)],
    validation_split=0.1,
    shuffle=True
)

model.save(os.path.join(MODEL_PATH, MODEL_NAME + SUFFIX + ".h5"))

Epoch 1/200


2022-04-10 16:19:46.513537: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1160455680 exceeds 10% of free system memory.
2022-04-10 16:19:47.211994: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1160455680 exceeds 10% of free system memory.


1771/1771 [==============================] - 7s 3ms/step - loss: 11.6516 - val_loss: 4.4110
Epoch 2/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.9982 - val_loss: 3.9497
Epoch 3/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.7867 - val_loss: 3.8487
Epoch 4/200
1771/1771 [==============================] - 5s 3ms/step - loss: 3.6622 - val_loss: 3.7405
Epoch 5/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.5599 - val_loss: 3.6115
Epoch 6/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.4962 - val_loss: 3.5536
Epoch 7/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.4393 - val_loss: 3.5455
Epoch 8/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.4022 - val_loss: 3.4669
Epoch 9/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.3781 - val_loss: 3.4925
Epoch 10/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.3469

Epoch 80/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.0261 - val_loss: 3.1052
Epoch 81/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.0272 - val_loss: 3.1177
Epoch 82/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.0224 - val_loss: 3.0986
Epoch 83/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.0211 - val_loss: 3.1278
Epoch 84/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.0201 - val_loss: 3.1136
Epoch 85/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.0165 - val_loss: 3.1025
Epoch 86/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.0134 - val_loss: 3.0924
Epoch 87/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.0120 - val_loss: 3.1025
Epoch 88/200
1771/1771 [==============================] - 6s 3ms/step - loss: 3.0108 - val_loss: 3.1126
Epoch 89/200
1771/1771 [==============================] - 6s 3ms

In [16]:
#!pip install selenium

In [27]:
plot_loss_per_epoch(
    history, model_name=MODEL_NAME, file_name=os.path.join(IMAGE_PATH, "model_loss.svg")
)

In [28]:
recon_errors = []

for index in tqdm(range(len(test_files))):
    signal, sr = load_sound_file(test_files[index])

    features = extract_signal_features(
        signal, sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, frames=frames
    )

    predictions = model.predict(features)
    # https://medium.com/nothingaholic/understanding-the-mean-squared-error-df41e2c87958
    mse = np.mean(np.mean(np.square(features - predictions), axis=1))
    recon_errors.append(mse)

  0%|          | 0/2290 [00:00<?, ?it/s]

In [32]:
print(recon_errors)

[4.8422604, 7.9922647, 10.207907, 2.6027083, 6.6915565, 6.81046, 3.6575277, 3.9191198, 6.700976, 2.840257, 5.5657187, 6.8733993, 5.4667645, 2.950575, 3.6040967, 2.7100592, 8.676019, 2.660546, 2.8983803, 4.1134124, 5.7321687, 7.9599137, 3.4800594, 3.6989427, 4.7616625, 4.790898, 2.5556192, 9.640824, 2.757738, 4.4364133, 2.637026, 2.8421962, 2.5229213, 3.179616, 2.6579394, 3.1941044, 2.7136858, 2.7867208, 3.6213994, 4.790846, 2.9191017, 9.42197, 2.500094, 7.9955926, 9.879132, 2.9511497, 4.8566933, 6.1030173, 2.7756326, 2.7359328, 4.721598, 9.004585, 4.417551, 3.0913026, 3.6010845, 7.6159306, 4.024854, 3.6407766, 4.201563, 3.1617894, 3.7488089, 3.0446734, 5.0119095, 3.7829525, 2.8787625, 2.7416782, 3.3354287, 6.081087, 4.3591437, 3.668041, 2.3749158, 3.3217869, 7.445009, 3.0389977, 3.4465258, 3.8080683, 3.560411, 4.9076066, 2.9769728, 2.5753398, 9.013907, 6.0687337, 13.0038, 9.306537, 4.144453, 3.4903395, 4.0178475, 3.0205643, 6.1388526, 3.354677, 2.6613154, 2.5986285, 2.4520774, 5.923625

In [36]:
print(len(features[0]))

320


# Model Evaluation

In [37]:
stack = np.column_stack((range(len(recon_errors)), recon_errors))
score_false = stack[test_labels == 0][:, 1]
score_true = stack[test_labels == 1][:, 1]

plot_histogram_by_class(
    score_false,
    score_true,
    bins=[20, 30],
    model_name=MODEL_NAME,
    file_name=os.path.join(IMAGE_PATH, "recon_error_dist.svg"),
)

In [38]:
THRESHOLD_MIN = 2.0
THRESHOLD_MAX = 7.0

p = figure(
    plot_width=600,
    plot_height=400,
    title=f"{MODEL_NAME}: Threshold Range Exploration",
    x_axis_label="Samples",
    y_axis_label="Reconstruction Error",
)

source = ColumnDataSource(
    dict(index=stack[test_labels == 0][:, 0], error=stack[test_labels == 0][:, 1])
)
p.scatter(
    "index",
    "error",
    fill_alpha=0.6,
    fill_color="crimson",
    line_color=None,
    legend_label="Normal Signals",
    source=source,
)

source = ColumnDataSource(
    dict(index=stack[test_labels == 1][:, 0], error=stack[test_labels == 1][:, 1])
)
p.scatter(
    "index",
    "error",
    fill_alpha=0.6,
    fill_color="indigo",
    line_color=None,
    legend_label="Abnormal Signals",
    source=source,
)

source = ColumnDataSource(
    data=dict(
        index=stack[:, 0],
        threshold_min=np.repeat(THRESHOLD_MIN, stack.shape[0]),
        threshold_max=np.repeat(THRESHOLD_MAX, stack.shape[0]),
    )
)

box = BoxAnnotation(
    bottom=THRESHOLD_MIN,
    top=THRESHOLD_MAX,
    fill_alpha=0.1,
    fill_color="magenta",
    line_color="darkmagenta",
    line_width=1.0,
)
p.add_layout(box)

p.legend.label_text_font_size = "8pt"
p.legend.location = "top_right"
p.title.align = "center"
p.title.text_font_size = "12pt"

p.add_tools(HoverTool(tooltips=[("index", "@index"), ("error", "@error")]))
show(p)

p.output_backend = "svg"
_ = export_svgs(p, filename=os.path.join(IMAGE_PATH, "thr_range_exp.svg"))

In [39]:
THRESHOLD_STEP = 0.2
thresholds = np.arange(THRESHOLD_MIN, THRESHOLD_MAX + THRESHOLD_STEP, THRESHOLD_STEP)
errors = []

for threshold in thresholds:
    predictions = get_prediction(stack[:, 1], threshold=threshold)
    conf_mat = confusion_matrix(test_labels, predictions)
    errors.append([threshold, conf_mat[1, 0], conf_mat[0, 1]])

errors = np.array(errors)

In [40]:
p = figure(
    plot_width=600,
    plot_height=400,
    title=f"{MODEL_NAME}: Best Threshold Exploration",
    x_axis_label="Reconstruction Error Threshold (%)",
    y_axis_label="# Samples",
)

source = ColumnDataSource(
    data=dict(
        threshold=errors[:, 0], false_negative=errors[:, 1], false_positive=errors[:, 2]
    )
)

p.line(
    x="threshold",
    y="false_negative",
    color="crimson",
    legend_label="False Negative",
    source=source,
)
p.circle(
    x="threshold", y="false_negative", color="crimson", fill_color="white", source=source
)
p.line(
    x="threshold",
    y="false_positive",
    color="indigo",
    legend_label="False Positive",
    source=source,
)
p.circle(
    x="threshold", y="false_positive", color="indigo", fill_color="white", source=source
)

p.legend.label_text_font_size = "8pt"
p.legend.location = "top_left"
p.legend.click_policy = "hide"
p.title.align = "center"
p.title.text_font_size = "12pt"

p.add_tools(
    HoverTool(
        tooltips=[
            ("threshold", "@threshold"),
            ("false_negative", "@false_negative"),
            ("false_positive", "@false_positive"),
        ]
    )
)
show(p)

p.output_backend = "svg"
_ = export_svgs(p, filename=os.path.join(IMAGE_PATH, "best_thr_exp.svg"))

In [44]:
THRESHOLD = 3.4
predictions = get_prediction(stack[:, 1], threshold=THRESHOLD)

plot_confusion_matrix(
    confusion_matrix(test_labels, predictions),
    model_name=MODEL_NAME,
    file_name=os.path.join(IMAGE_PATH, "conf_mat.svg"),
)

In [45]:
print(
    f"Accuracy: {accuracy_score(test_labels, predictions):.2%}, \
Precision: {precision_score(test_labels, predictions):.2%}, \
Recall: {recall_score(test_labels, predictions):.2%}, \
F1: {f1_score(test_labels, predictions):.2%}"
)

Accuracy: 88.12%, Precision: 90.84%, Recall: 90.71%, F1: 90.77%


In [46]:
plot_roc_curve(
    roc_curve(test_labels, recon_errors),
    roc_auc_score(test_labels, recon_errors),
    model_name=MODEL_NAME,
    file_name=os.path.join(IMAGE_PATH, "roc_curve.svg"),
)

In [47]:
plot_pr_curve(
    precision_recall_curve(test_labels, recon_errors),
    average_precision_score(test_labels, recon_errors),
    model_name=MODEL_NAME,
    file_name=os.path.join(IMAGE_PATH, "pr_curve.svg"),
)